In [1]:
import numpy as np  
import pandas as pd
from pandas import Series, DataFrame
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.python.ops import confusion_matrix
from tensorflow.python.ops import math_ops

import matplotlib.pyplot as plt

C:\Users\35266\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
#Data=pd.read_csv('C:/Users/dkoch/Documents/tensorflow/R_Input.csv')
Data=pd.read_csv('R_Input.csv')
Data=Data.fillna(0)
Data['dprio']=Data['dprio']/(10*np.std(Data['dprio']))

Data['dprio0']=Data['dprio']**0
Data['dprio2']=Data['dprio']**2
Data['dprio3']=Data['dprio']**3
Data=Data.rename(columns={'dprio': 'dprio1'})
Data['pax_total']=Data['pax_total']-Data['group_pax']
Data['pax_total']=Data['pax_total'].apply(lambda x: x if x>0 else 0)

direction=pd.get_dummies(Data['direction'],drop_first=True)
direction.columns=['direction:'+x for x in direction.columns]

whichholiday_left_LUX=pd.get_dummies(Data['whichholiday_left_LUX'],drop_first=True)
whichholiday_left_LUX.columns=['whichholiday_left_LUX:'+x for x in whichholiday_left_LUX.columns]

whichholiday_right_LUX=pd.get_dummies(Data['whichholiday_right_LUX'],drop_first=True)
whichholiday_right_LUX.columns=['whichholiday_right_LUX:'+x for x in whichholiday_right_LUX.columns]

Data=pd.concat([Data,direction,whichholiday_left_LUX,whichholiday_right_LUX],axis=1)

Data.index=Data['calendar_date']+'_'+Data['direction']

Data['weekday']=Data['calendar_date'].apply(lambda x: pd.to_datetime(x).weekday()+1)
Data['yearweek']=Data['calendar_date'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])

In [3]:
dprio=Data[['dprio'+str(i) for i in range(0,4)]].drop_duplicates().transpose()
dprio.columns=range(365,-1,-1)
dprio=dprio[[x for x  in range(0,366)]]

X_data=Data.drop_duplicates(subset=['calendar_date','direction'])[[x for x in Data.columns if 'direction:' in x]\
+['weekday','yearweek']\
+[x for x in Data.columns if 'whichholiday_left_LUX:' in x]\
+[x for x in Data.columns if 'whichholiday_right_LUX:' in x]\
+['holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

In [4]:
X_data=X_data[['direction:O','weekday','yearweek',
'holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

Data['concat']=Data['calendar_date']+'_'+Data['direction']
y_data=Data[['concat','dprio1','pax_total']].pivot(index='concat',columns='dprio1',values='pax_total')

In [5]:
X_test=X_data.loc[[x for x in X_data.index if '2019' in x],:]
X_test=X_test.values
dprio=dprio.values

In [6]:
sess=tf.Session()
saver=tf.train.import_meta_graph('mymodel.meta')
saver.restore(sess, tf.train.latest_checkpoint('./'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./mymodel


In [7]:
graph = tf.get_default_graph()

In [8]:
[n.name for n in tf.get_default_graph().as_graph_def().node if 'Variable' in n.name]

['Variable',
 'Variable/Assign',
 'Variable/read',
 'Variable_1',
 'Variable_1/Assign',
 'Variable_1/read',
 'Variable_2',
 'Variable_2/Assign',
 'Variable_2/read',
 'Variable_3',
 'Variable_3/Assign',
 'Variable_3/read',
 'GradientDescent/update_Variable/ApplyGradientDescent',
 'GradientDescent/update_Variable_1/ApplyGradientDescent',
 'GradientDescent/update_Variable_2/ApplyGradientDescent',
 'GradientDescent/update_Variable_3/ApplyGradientDescent']

In [9]:
X = graph.get_tensor_by_name("Placeholder:0")
Z = graph.get_tensor_by_name("Placeholder_1:0")

In [10]:
feed_dict={X:X_test,Z:dprio.reshape(4,366)}

In [11]:
w1 = graph.get_tensor_by_name("Variable:0")
w2 = graph.get_tensor_by_name("Variable_1:0")
b1 = graph.get_tensor_by_name("Variable_2:0")
b2 = graph.get_tensor_by_name("Variable_3:0")

In [12]:
layer_1=sess.run(tf.nn.sigmoid(tf.add(tf.matmul(X, w1), b1)),feed_dict)
layer_2=sess.run(tf.add(tf.matmul(layer_1, w2), b2),feed_dict)
out_layer=sess.run(tf.exp(tf.matmul(layer_2, Z)),feed_dict)

In [13]:
out_layer

array([[0.00703848, 0.00699848, 0.00695869, ..., 0.00081396, 0.00080904,
        0.00080416],
       [0.00689793, 0.00685882, 0.00681992, ..., 0.00080433, 0.00079951,
        0.00079471],
       [0.02901513, 0.02891239, 0.02881013, ..., 0.01131133, 0.01129532,
        0.01127944],
       ...,
       [0.09666863, 0.09639315, 0.09611911, ..., 0.05072637, 0.05068219,
        0.05063825],
       [0.00691603, 0.00688398, 0.00685206, ..., 0.00111846, 0.00111267,
        0.0011069 ],
       [0.0069153 , 0.00688325, 0.00685133, ..., 0.00111826, 0.00111246,
        0.0011067 ]], dtype=float32)